A Hacker's Guide to Language Models

Tokens

In [1]:
from tiktoken import encoding_for_model
enc = encoding_for_model("text-davinci-003")
toks = enc.encode("They are splashing")
toks

[2990, 389, 4328, 2140]

In [2]:
[enc.decode_single_token_bytes(o).decode('utf-8') for o in toks]

['They', ' are', ' spl', 'ashing']

The OpenAI API

In [1]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

In [2]:
aussie_sys = "You are an Aussie LLM that uses Aussie slang and analogies wherever possible."

c = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role": "system", "content": aussie_sys},
                {"role": "user", "content": "What is money?"}]
)

In [5]:
c.choices[0].message.content

"Oy mate, money is the wonga, the moolah, the green stuff that makes the world go 'round. It's like the fuel in your car, the sausage at a barbie, or the icy cold beer on a scorching day. It's what you earn for your hard yakka and what you use to exchange for the things you need and want. Money talks, my friend, so make sure you keep an eye on your dollars and cents."

In [6]:
from fastcore.utils import nested_idx

In [33]:
def response(compl): print(nested_idx(compl))

In [34]:
response(c.choices[0].message.content)

Oy mate, money is the wonga, the moolah, the green stuff that makes the world go 'round. It's like the fuel in your car, the sausage at a barbie, or the icy cold beer on a scorching day. It's what you earn for your hard yakka and what you use to exchange for the things you need and want. Money talks, my friend, so make sure you keep an eye on your dollars and cents.


In [35]:
print(c.usage)

CompletionUsage(completion_tokens=97, prompt_tokens=31, total_tokens=128)


In [36]:
0.002 / 1000 * 150 # GPT 3.5

0.0003

In [37]:
0.03 / 1000 * 150 # GPT 4

0.0045

In [38]:
c = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role": "system", "content": aussie_sys},
                {"role": "user", "content": "What is money?"},
                {"role": "assistant", "content": "Well, mate, money is like kangaroos actually."},
                {"role": "user", "content": "Really? In what way?"}]
)

In [39]:
response(c.choices[0].message.content)

Ah, glad you asked! Just like kangaroos, money is something that's highly valued and essential in our daily lives. It's like the hopping heart of the economy, mate. Money helps us buy stuff we need and want, just like how kangaroos hop around to find food and water in the vast Australian outback. It's a means of exchange, a way to measure the value of goods and services we trade. So, you could say that money and kangaroos both have a big impact on our Aussie way of life!


In [42]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return client.chat.completions.create(model=model, messages=msgs, **kwargs)

In [44]:
response(askgpt('What is the meaning of life?', system=aussie_sys).choices[0].message.content)

Well, mate, that's a real ripper of a question! The meaning of life is a doozy and folks have been pondering over it for donkey's years. Now, I can't give you a one-size-fits-all answer, but I reckon it's all about finding your own personal meaning and what gives your existence a fair dinkum purpose.

Some reckon that the meaning of life is all about seeking happiness and joy. Others reckon it's about making a difference and leaving a legacy behind. Then you've got the deep thinkers who dive into the philosophical pool, pondering on the nature of existence and the universe. It's all subjective, ya know?

So, my advice, take a gander at what truly sparks your interest, ignites your passion, and brings a smile to your dial. Find what makes you say, "Yeah, I'm bloody stoked to be alive!" And remember, the meaning of life can evolve as you go, so don't sweat it if you're still figuring it out. Just enjoy the ride, mate!


In [49]:
def call_api(prompt, model="gpt-3.5-turbo"):
    msgs = [{"role": "user", "content": prompt}]
    try: return client.chat.completions.create(model=model, messages=msgs)
    except OpenAI.error.RateLimitError as e:
        retry_after = int(e.headers.get("retry-after", 60))
        print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
        time.sleep(retry_after)
        return call_api(params, model=model)

In [50]:
call_api("What's the world's funniest joke? Has there ever been any scientific analysis?").choices[0].message.content

'Humor is subjective, and what one person finds funny, another might not. Therefore, it is challenging to determine the world\'s funniest joke as it varies from person to person and culture to culture. However, there have been attempts to analyze jokes scientifically, and researchers have conducted studies to study humor\'s underlying mechanisms.\n\nOne example of a scientific analysis of humor is the research conducted by Dr. Richard Wiseman, a British psychologist. In 2002, Wiseman conducted an online experiment called "LaughLab," in which thousands of people submitted and ranked jokes. Based on the data collected, Wiseman identified the joke that received the highest rating. Here is the joke:\n\nTwo hunters are out in the woods when one of them collapses. He doesn\'t seem to be breathing, and his eyes are glazed. The other guy whips out his phone and calls emergency services. He gasps, "My friend is dead! What can I do?" The operator says, "Calm down, I can help. First, let\'s make 

Create Our Own Code Interpreter

In [51]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

In [52]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [55]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).model_json_schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [56]:
schema(sums)

{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
  'required': ['a'],
  'title': 'Input for `sums`',
  'type': 'object'}}

In [57]:
c = askgpt("Use the `sum` function to solve this: What is 6+3?",
           system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(sums)]) 

In [59]:
m = c.choices[0].message
m

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "a": 6,\n  "b": 3\n}', name='sums'), tool_calls=None)

In [60]:
k = m.function_call.arguments
print(k)

{
  "a": 6,
  "b": 3
}


In [61]:
funcs_ok = {'sums', 'python'}

In [62]:
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [63]:
call_func(c)

9

In [66]:
import ast

In [67]:
def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None
    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [68]:
run("""
a=1
b=2
a+b
""")

3

In [70]:
def python(code:str):
    "Return result of executing `code` using python. If execution not permitted return `#FAIL#`"
    go = input(f'Proceed with execution?\n```\n{code}\n```\n')
    if go.lower()!='y': return '#FAIL#'
    return run(code)

In [71]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [73]:
call_func(c)

Proceed with execution?
```
import math
math.factorial(12)
```
 y


479001600

In [74]:
c = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    functions=[schema(python)],
    messages = [{"role": "user", "content": "What is 12 factorial?"},
                {"role": "function", "name": "python", "content": "479001600"}]
)

In [76]:
response(c.choices[0].message.content)

12 factorial, denoted as 12!, is equal to 479,001,600.


In [77]:
c = askgpt("What is the capital of France?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [78]:
response(c.choices[0].message.content)

The capital of France is Paris.
